# Packages

In [8]:
#from PIL import Image
#import sys
#sys.path.append('/usr/local/lib/python3.7/site-packages')
#import os
import numpy as np
import importlib
from numpy import linalg
import PO_helperfuns
from PO_helperfuns import *
import PO_DatFrameFuns
from PO_DatFrameFuns import *
import scipy.stats as scs
import matplotlib as mpl
import matplotlib.pyplot as plt
#importlib.reload(helperfuns)
import pandas as pd
import sympy as sp
from sympy import *
from sympy.abc import lamda, beta
from sympy.solvers import solve
#np.set_printoptions(precision=3, suppress = True)
#import seaborn as sns
# next two libraries are used to flatten list of lists
import functools
import operator
# for formatting tick labels
#from matplotlib.ticker import FormatStrFormatter

#for parallelizing:
import multiprocessing as mp
from pathos.multiprocessing import ProcessingPool as Pool




## Set up Parameter Grid

In [40]:

svals = np.array([0,0.5,1,2,3,3.5])
muvals = np.array([-2, -0.5, 0,0.5,1,2])
betavals = np.array([0,0.33, 0.66, 1])
df = get_param_grid(svals, muvals, betavals)

## We use parallelization to to greatly speed up the dataframe generation. 

In [41]:
# Parallelize and find equilibria

cores=mp.cpu_count()
# split dataframe over cores, stored in df_split
df_split = np.array_split(df, cores, axis=0)

# remove this line... just for testing
pool = Pool(cores)

# Run get_1side_df over each dataframe in df_split. 
# get_1side_df iterates 50000 time steps. Stack dataframe (df) parts into one df.

df_out = np.vstack(pool.map(get_1side_df, df_split))
pool.close()
pool.join()
pool.clear()
df_1side = pd.DataFrame(df_out, columns = df.columns)

# fix the column names
colnames = ['mu', 'K', 'pc', 's', 'beta',
       'u1init', 'u2init', 'buinit', 'r1init', 'r2init', 'u1eq', 'u2eq',
       'bueq', 'r1eq', 'r2eq', 'Weq', 'time', 'reached_eq', 'URstable']
df_1side = df_1side[colnames] # gets rid of the added unnamed columns
#save to csv
df_1side.to_csv('data_1side_1stRound.csv')

## Check the result of the 500,000th iteration to see if it is at equilibrium.

In [42]:
df_1side = pd.read_csv('data_1side_1stRound.csv')
df_1side['difference'] = np.abs(df_1side.u1eq - df_1side.u2eq)

In [43]:
df_1side.query('difference > 1e-12')

,Unnamed: 0,mu,K,pc,s,beta,u1init,u2init,buinit,r1init,...,u1eq,u2eq,bueq,r1eq,r2eq,Weq,time,reached_eq,URstable,difference
270,270,-2.0,0.499968,3.167124e-05,2.0,0.0,0.05,0.1,0.85,0.05,...,0.333333,0.333333,0.333333,1.0,1.0,1.500000,500000,True,0.0,1.752765e-12
276,276,-2.0,0.499968,3.167124e-05,2.0,0.0,0.05,0.1,0.85,0.50,...,0.333333,0.333333,0.333333,1.0,1.0,1.500000,500000,True,0.0,1.752765e-12
282,282,-2.0,0.499968,3.167124e-05,2.0,0.0,0.05,0.1,0.85,0.85,...,0.333333,0.333333,0.333333,1.0,1.0,1.500000,500000,True,0.0,1.752765e-12
288,288,-2.0,0.499968,3.167124e-05,2.0,0.0,0.05,0.1,0.85,0.20,...,0.333333,0.333333,0.333333,1.0,1.0,1.500000,500000,True,0.0,1.752765e-12
294,294,-2.0,0.499968,3.167124e-05,2.0,0.0,0.05,0.1,0.85,0.90,...,0.333333,0.333333,0.333333,1.0,1.0,1.500000,500000,True,0.0,1.752154e-12
300,300,-2.0,0.499968,3.167124e-05,2.0,0.0,0.48,0.4,0.12,0.05,...,0.333333,0.333333,0.333333,1.0,1.0,1.500000,500000,True,0.0,1.752765e-12
306,306,-2.0,0.499968,3.167124e-05,2.0,0.0,0.48,0.4,0.12,0.50,...,0.333333,0.333333,0.333333,1.0,1.0,1.500000,500000,True,0.0,1.752154e-12
312,312,-2.0,0.499968,3.167124e-05,2.0,0.0,0.48,0.4,0.12,0.85,...,0.333333,0.333333,0.333333,1.0,1.0,1.500000,500000,True,0.0,1.752154e-12
318,318,-2.0,0.499968,3.167124e-05,2.0,0.0,0.48,0.4,0.12,0.20,...,0.333333,0.333333,0.333333,1.0,1.0,1.500000,500000,True,0.0,1.752765e-12
324,324,-2.0,0.499968,3.167124e-05,2.0,0.0,0.48,0.4,0.12,0.90,...,0.333333,0.333333,0.333333,1.0,1.0,1.500000,500000,True,0.0,1.752154e-12


In [16]:
u1 = df_1side.u1eq; beta = df_1side.beta; pc = df_1side.pc; K = df_1side.K
should_be_0 = -2*beta*u1**2*(K+pc) + u1*(2 + (K+pc)*(2+beta))-2*(K+pc)
df

In [19]:
df_1side.loc[should_be_0>1e-10,:]

,Unnamed: 0,mu,K,pc,s,beta,u1init,u2init,buinit,r1init,...,u1eq,u2eq,bueq,r1eq,r2eq,Weq,time,reached_eq,URstable,difference
245,245,-2.0,0.841344,2.866516e-07,3.0,0.0,0.05,0.1,0.85,0.90,...,0.605131,0.308707,0.086163,1.0,1.0,1.841345,500000,False,0.0,0.296424
255,255,-2.0,0.841344,2.866516e-07,3.0,0.0,0.48,0.4,0.12,0.50,...,0.585369,0.328468,0.086163,1.0,1.0,1.841345,500000,False,0.0,0.256900
260,260,-2.0,0.841344,2.866516e-07,3.0,0.0,0.48,0.4,0.12,0.85,...,0.481003,0.432834,0.086163,1.0,1.0,1.841345,500000,False,0.0,0.048169
270,270,-2.0,0.841344,2.866516e-07,3.0,0.0,0.48,0.4,0.12,0.90,...,0.721847,0.191990,0.086163,1.0,1.0,1.841345,500000,False,0.0,0.529857


Save the weird equilibrix

In [45]:
df_weird = df_1side.loc[should_be_0>1e-10,:]
df_weird.to_csv('df_weird.csv')

KeyError: True

# Reflect the dataframe

In [44]:
# note: once i re-run with the fixed check of if it reached eq, this cell won't be necessary
df_1side.loc[~where_wrong, 'reached_eq'] = True
df_1side.loc[where_wrong,'reached_eq'] = False


IndexingError: (0       True
1       True
2       True
3       True
4       True
5       True
6       True
7       True
8       True
9       True
10      True
11      True
12      True
13      True
14      True
15      True
16      True
17      True
18      True
19      True
20      True
21      True
22      True
23      True
24      True
25      True
26      True
27      True
28      True
29      True
        ... 
1170    True
1171    True
1172    True
1173    True
1174    True
1175    True
1176    True
1177    True
1178    True
1179    True
1180    True
1181    True
1182    True
1183    True
1184    True
1185    True
1186    True
1187    True
1188    True
1189    True
1190    True
1191    True
1192    True
1193    True
1194    True
1195    True
1196    True
1197    True
1198    True
1199    True
Length: 1200, dtype: bool, 'reached_eq')

In [ ]:

df_total = reflect_df(df_1side)
df = df_total.sort_values(by=['mu','beta','s'])
df.reset_index(inplace=True, drop=True) 
colnames = ['mu', 'K', 'pc', 's', 'beta',
       'u1init', 'u2init', 'buinit', 'r1init', 'r2init', 'u1eq', 'u2eq',
       'bueq', 'r1eq', 'r2eq', 'Weq', 'time', 'reached_eq', 'URstable']
df = df[colnames]
df.to_csv('data.csv')

# Find unique equilibria and analyze them

In [34]:
def get_UniqueEquilibria(df,if_save=False):
    df_eq = df.round(6)[df.reached_eq==True].groupby(['K','pc','s','mu','beta','u1eq','u2eq','bueq',
                                                     'r1eq','r2eq','Weq','URstable'], as_index = False)
    df_eq = df_eq['u2init'].count()
    df_eq.rename(columns={'u2init':'NumInitials'}, inplace=True)
    # df_eq.reset_index(inplace=True, drop=True)
    # Cs = Grad_s([df_eq.u1eq, df_eq.u2eq], [df_eq.r1eq, df_eq.r2eq],df_eq.Weq, df_eq.s, df_eq.beta, df_eq.mu)
    # df_eq['C_s'] = Cs
    if if_save:
        df_eq.to_csv('UniqueEquilibria.csv', index = False)
    return(df_eq)

# We have a lot of repeat equilibria in the dataframe df because many of the initial points go to the same equilibrium.
# We now extract the unique equilibria for each parameter combination.
df = pd.read_csv('data.csv')
df_main = df[(df.reached_eq==1)]
unique_eq = get_UniqueEquilibria(df_main)
diff = np.abs(unique_eq.u1eq - unique_eq.u2eq)
unique_eq['difference'] = diff
unique_eq.to_csv('UniqueEquilibriaDF.csv')

In [35]:
unique_eq['x_pos_invades'] = True
unique_eq['x_neg_invades'] = False

In [21]:
unique_eq = pd.read_csv('UniqueEquilibriaDF.csv')


In [36]:
#unique_eq = pd.read_csv('UniqueEquilibriaDF.csv')

cores=mp.cpu_count()
df_split = np.array_split(unique_eq, cores, axis=0)
pool = Pool(cores)
df_out = np.vstack(pool.map(df_ext_stability_iterate, df_split))
pool.close()
pool.join()
pool.clear()


In [37]:
cols = unique_eq.columns
df_fixed = pd.DataFrame(df_out, columns = cols)

In [38]:
df_fixed.query('x_neg_invades==True')

,K,pc,s,mu,beta,u1eq,u2eq,bueq,r1eq,r2eq,Weq,URstable,NumInitials,difference,x_pos_invades,x_neg_invades
63,0.841344,0.158655,3,2,0,0.5,0.5,0,1,1,2,0,30,0,True,True
64,0.841344,0.158655,3,2,0.33,0.5,0.5,0,0.835,0.835,1.835,0,30,0,True,True
65,0.841344,0.158655,3,2,0.66,0.5,0.5,0,0.67,0.67,1.67,0,30,0,True,True
66,0.841344,0.158655,3,2,1,0.5,0.5,0,0.5,0.5,1.5,0,30,0,True,True


Now we check that the values of $C_s$ reflect whether invasion actually happens.

In [39]:
df.query('mu==2 & s == 3')

,Unnamed: 0,mu,K,pc,s,beta,u1init,u2init,buinit,r1init,r2init,u1eq,u2eq,bueq,r1eq,r2eq,Weq,time,reached_eq,URstable
2010,2010,2.0,0.841344,0.158655,3.0,0.0,0.05,0.10,0.85,0.05,0.10,0.5,0.5,1.433258e-07,1.0,1.0,2.0,500000,True,0.0
2011,2011,2.0,0.841344,0.158655,3.0,0.0,0.05,0.10,0.85,0.50,0.30,0.5,0.5,1.433258e-07,1.0,1.0,2.0,500000,True,0.0
2012,2012,2.0,0.841344,0.158655,3.0,0.0,0.05,0.10,0.85,0.85,0.90,0.5,0.5,1.433258e-07,1.0,1.0,2.0,500000,True,0.0
2013,2013,2.0,0.841344,0.158655,3.0,0.0,0.05,0.10,0.85,0.20,0.89,0.5,0.5,1.433258e-07,1.0,1.0,2.0,500000,True,0.0
2014,2014,2.0,0.841344,0.158655,3.0,0.0,0.05,0.10,0.85,0.90,0.20,0.5,0.5,1.433258e-07,1.0,1.0,2.0,500000,True,0.0
2015,2015,2.0,0.841344,0.158655,3.0,0.0,0.48,0.40,0.12,0.05,0.10,0.5,0.5,1.433258e-07,1.0,1.0,2.0,500000,True,0.0
2016,2016,2.0,0.841344,0.158655,3.0,0.0,0.48,0.40,0.12,0.50,0.30,0.5,0.5,1.433258e-07,1.0,1.0,2.0,500000,True,0.0
2017,2017,2.0,0.841344,0.158655,3.0,0.0,0.48,0.40,0.12,0.85,0.90,0.5,0.5,1.433258e-07,1.0,1.0,2.0,500000,True,0.0
2018,2018,2.0,0.841344,0.158655,3.0,0.0,0.48,0.40,0.12,0.20,0.89,0.5,0.5,1.433258e-07,1.0,1.0,2.0,500000,True,0.0
2019,2019,2.0,0.841344,0.158655,3.0,0.0,0.48,0.40,0.12,0.90,0.20,0.5,0.5,1.433258e-07,1.0,1.0,2.0,500000,True,0.0


In [32]:
K = Kfun(3, scs.norm(2))
pc = pcfun(3,scs.norm(2))
beta = 0
u1 = 0.5
should_be_0 = -2*beta*u1**2*(K+pc) + u1*(2 + (K+pc)*(2+beta))-2*(K+pc)

In [33]:
should_be_0

2.866515720345575e-07

In [17]:
# find examples that don't agree with the sign of C_s
df_disagree_x_neg = df_fixed.query('C_s < 0 & x_neg_invades == False')
#max(np.abs(df_disagree_x_neg.C_s))

In [20]:
df_disagree_x_neg

,Unnamed: 0,K,pc,s,mu,beta,u1eq,u2eq,bueq,r1eq,r2eq,Weq,URstable,NumInitials,C_s,difference,x_pos_invades,x_neg_invades
0,0,0.0,0.02275,0.0,-2.0,0.0,0.042593,0.042593,0.914813,1.0,1.0,1.06825,0.0,30,-0.050542,0.0,False,False
1,1,0.0,0.02275,0.0,-2.0,0.33,0.042321,0.042321,0.915358,0.986034,0.986034,1.067615,0.0,30,-0.050572,0.0,False,False
2,2,0.0,0.02275,0.0,-2.0,0.66,0.042052,0.042052,0.915896,0.972246,0.972246,1.066988,0.0,30,-0.050601,0.0,False,False
3,3,0.0,0.02275,0.0,-2.0,1.0,0.041778,0.041778,0.916444,0.958222,0.958222,1.066349,0.0,30,-0.050632,0.0,False,False
4,4,0.0,0.308538,0.0,-0.5,0.0,0.320456,0.320456,0.359087,1.0,1.0,1.925613,0.0,30,-0.182833,0.0,False,False
5,5,0.0,0.308538,0.0,-0.5,0.33,0.314284,0.314284,0.371432,0.896286,0.896286,1.861613,0.0,30,-0.189118,0.0,False,False
6,6,0.0,0.308538,0.0,-0.5,0.66,0.307948,0.307948,0.384104,0.796754,0.796754,1.800195,0.0,30,-0.195571,0.0,False,False
7,7,0.0,0.308538,0.0,-0.5,1.0,0.30127,0.30127,0.397459,0.69873,0.69873,1.739706,0.0,30,-0.202371,0.0,False,False
8,8,0.0,0.5,0.0,0.0,0.0,0.4,0.4,0.2,1.0,1.0,2.5,0.0,30,-0.159577,0.0,False,False
9,9,0.0,0.5,0.0,0.0,0.33,0.394506,0.394506,0.210987,0.869813,0.869813,2.369813,0.0,30,-0.168343,0.0,False,False


We confirmed that the equation to find u1 = u2 equilibria works. Now we want to find parameter values for which the equilibrium is unstable or there are cycles (complex eigenvalues)

In [27]:
u1, r1, K, pc, L,R, W = sp.symbols('\hat{u}_1, \hat{r}_1,  K, \pi_C, L(1/2),R, \hat{W}')
from sympy.abc import beta, lamda
# R = 1 + r1
#W = 1 + pc + 2*r1*L
A = K*R/(4*W*u1)
B = (L*(1-u1))/W
C = (-u1*L)/W
D = -beta*r1/R
E = 1/R

J = Matrix([[A,-A,B,C],[-A,A,C,B],[D,0,E,0],[0,D,0,E]])

In [8]:
charpoly = factor(J.charpoly(lamda).as_expr())
charpoly

(2*L(1/2)*\hat{r}_1*\hat{u}_1*beta - L(1/2)*\hat{r}_1*beta - R*\hat{W}*lamda**2 + \hat{W}*lamda)*(K*R**2*lamda - K*R - 2*L(1/2)*\hat{r}_1*\hat{u}_1*beta - 2*R*\hat{W}*\hat{u}_1*lamda**2 + 2*\hat{W}*\hat{u}_1*lamda)/(2*R**2*\hat{W}**2*\hat{u}_1)

In [10]:
n,d = fraction(charpoly)
charpoly = n
print(latex(charpoly.subs([(R,1+r1)])))

\left(2 L(1/2) \hat{r}_1 \hat{u}_1 \beta - L(1/2) \hat{r}_1 \beta - \hat{W} \lambda^{2} \left(\hat{r}_1 + 1\right) + \hat{W} \lambda\right) \left(K \lambda \left(\hat{r}_1 + 1\right)^{2} - K \left(\hat{r}_1 + 1\right) - 2 L(1/2) \hat{r}_1 \hat{u}_1 \beta - 2 \hat{W} \hat{u}_1 \lambda^{2} \left(\hat{r}_1 + 1\right) + 2 \hat{W} \hat{u}_1 \lambda\right)


In [11]:
evals = J.eigenvals(lamda)
evals = list(evals)

In [16]:
evals[2]

(K*R**2 + 2*\hat{W}*\hat{u}_1 + sqrt(K**2*R**4 - 4*K*R**2*\hat{W}*\hat{u}_1 - 16*L(1/2)*R*\hat{W}*\hat{r}_1*\hat{u}_1**2*beta + 4*\hat{W}**2*\hat{u}_1**2))/(4*R*\hat{W}*\hat{u}_1)